## AML-3204 Social Media Analytics 

## Project Title: Collaborative Filtering-based vs Hybrid Recommender System

### Group Members

- SAI VARUN KOLLIPARA - C0828403
- PRAMOD REDDY GURRALA – C0850493
- DEEKSHA NAIKAP – C0835440
- BHANU PRAKASH MAHADEVUNI – C0850515

## 1. Importing Packages

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tweepy
import re
import emoji
import nltk
nltk.download('words')
words = set(nltk.corpus.words.words())
stopword = nltk.corpus.stopwords.words('english')

import glob
import string
import math

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\gruha\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


## 2. Import the Movie Data

In [2]:
movieInfoFile = 'data_files/u.item'
movieInfo = pd.read_csv(movieInfoFile, sep="|", header=None, index_col=False,
                  names=['itemId','title'], usecols=[0,1])
movieInfo

,itemId,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)
...,...,...
1677,1678,Mat' i syn (1997)
1678,1679,B. Monkey (1998)
1679,1680,Sliding Doors (1998)
1680,1681,You So Crazy (1994)


In [3]:
movieInfo['year'] = movieInfo.title.str.extract('(\(\d\d\d\d\))',expand=False)
movieInfo['year'] = movieInfo.year.str.extract('(\d\d\d\d)',expand=False)

#Removing the years from the 'title' column
movieInfo['title'] = movieInfo.title.str.replace('(\(\d\d\d\d\))', '')

C:\Users\gruha\AppData\Local\Temp/ipykernel_13176/3145695913.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  movieInfo['title'] = movieInfo.title.str.replace('(\(\d\d\d\d\))', '')


In [4]:
movieInfo

,itemId,title,year
0,1,Toy Story,1995
1,2,GoldenEye,1995
2,3,Four Rooms,1995
3,4,Get Shorty,1995
4,5,Copycat,1995
...,...,...,...
1677,1678,Mat' i syn,1997
1678,1679,B. Monkey,1998
1679,1680,Sliding Doors,1998
1680,1681,You So Crazy,1994


## 2.1 Data Pre-Processing

Extract the keywords from the movie list, by removing the numbers and symbols.

In [5]:
movieInfo['tagword'] = movieInfo['title'].str.lower()
movieInfo['tagword'] = movieInfo.tagword.str.replace(' ', '')
movieInfo['tagword'] = movieInfo['tagword'].apply(lambda x: "{}{}".format('#', x))
movieInfo['tagword'] = movieInfo.tagword.str.replace('(\([A-Za-z0-9_]+\))', '')
movieInfo

C:\Users\gruha\AppData\Local\Temp/ipykernel_13176/3564921589.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  movieInfo['tagword'] = movieInfo.tagword.str.replace('(\([A-Za-z0-9_]+\))', '')


,itemId,title,year,tagword
0,1,Toy Story,1995,#toystory
1,2,GoldenEye,1995,#goldeneye
2,3,Four Rooms,1995,#fourrooms
3,4,Get Shorty,1995,#getshorty
4,5,Copycat,1995,#copycat
...,...,...,...,...
1677,1678,Mat' i syn,1997,#mat'isyn
1678,1679,B. Monkey,1998,#b.monkey
1679,1680,Sliding Doors,1998,#slidingdoors
1680,1681,You So Crazy,1994,#yousocrazy


In [6]:
movieInfo['title'][189]

'Henry V '

## 3. Movie Tweets Extraction

### 3.1 My API Keys

In [38]:
api_key = "LN8F7skyqFBFktr9GPODmXIaq"
api_secret = "LhWGHFC34yMkWv36plDO9hFpoZbindGiMIhwmW7dNY04ZMXk2g"
access_token = "1528781275615662083-6AEJcXfK3XtgA18oknE2UoofqDgug4"
access_secret = "nIPqOwmqMEXBhwYD0wVQKENC4jWWvjU71HlBH9ZtnH2rv"

'''
api_key = "xuwJ0lqiFIstrNutasxpW1IZo"
api_secret = "AXcDeHTofmwS1dhkI2EzVradtEhcBkH5m9vVX913w9VwIYO6KO"
access_token = "1530292039610421249-KVozftsjhgdqxgkZ8ze0Er3P432Izy"
access_secret = "c43MIyZdMOQ2zfjawOzw3QjREOGyky4OTXtWB9LciBXuN"
'''

'''
api_key = "v6jJwDji986CQpaKKcFguRWFu"
api_secret = "3u3OZsf4In7ee7nbivkWqXmMXwRpY0Pmv3teDM6qCXhfHihleG"
access_token = "1530281103193423872-DYsr64HT2PbTvzXWLaZQOjnn5843fd"
access_secret = "Iz7WPgYJyC94bz2is6tXq1nBVk7biC2DVIQmKFXTCtcpO"
'''

'\napi_key = "v6jJwDji986CQpaKKcFguRWFu"\napi_secret = "3u3OZsf4In7ee7nbivkWqXmMXwRpY0Pmv3teDM6qCXhfHihleG"\naccess_token = "1530281103193423872-DYsr64HT2PbTvzXWLaZQOjnn5843fd"\naccess_secret = "Iz7WPgYJyC94bz2is6tXq1nBVk7biC2DVIQmKFXTCtcpO"\n'

In [39]:
auth = tweepy.OAuthHandler(api_key, api_secret)
auth.set_access_token(access_token, access_secret)
api = tweepy.API(auth)

### 3.2 Function to Scrap the Data

In [9]:
# function to perform data extraction
def scrape(movieID, movie_title, words):
    numtweet = 100  
    
    # Creating DataFrame using pandas
    db = pd.DataFrame(columns=['Movie Id', 'Title', 'Tagword', 'description', 'totaltweets', 'retweetcount', 'text', 'hashtags'])
      
    # We are using .Cursor() to search through twitter for the required tweets.
    # The number of tweets can be restricted using .items(number of tweets)
    tweets = tweepy.Cursor(api.search_tweets, q=words, lang="en",
                           tweet_mode='extended').items(numtweet)
     
    # .Cursor() returns an iterable object. Each item in 
    # the iterator has various attributes that you can access to 
    # get information about each tweet
    list_tweets = [tweet for tweet in tweets]
      
    # Counter to maintain Tweet Count
    i = 1  
      
    # we will iterate over each tweet in the list for extracting information about each tweet
    for tweet in list_tweets:
        movie_id = movieID
        title = movie_title
        tagword = words
        description = tweet.user.description
        totaltweets = tweet.user.statuses_count
        retweetcount = tweet.retweet_count
        hashtags = tweet.entities['hashtags']
          
        # Retweets can be distinguished by a retweeted_status attribute,
        # in case it is an invalid reference, except block will be executed
        try:
            text = tweet.retweeted_status.full_text
        except AttributeError:
            text = tweet.full_text
        hashtext = list()
        for j in range(0, len(hashtags)):
            hashtext.append(hashtags[j]['text'])
          
        # Here we are appending all the extracted information in the DataFrame
        ith_tweet = [movie_id, title, tagword, description,
                     totaltweets, retweetcount, text, hashtext]
        db.loc[len(db)] = ith_tweet
          
        # Function call to print tweet data on screen
#         print(ith_tweet)
        i = i+1
    filename = str(movie_title) + '_tweets.csv'
      
    # we will save our database as a CSV file.
    db.to_csv('movie_tweets_collected/' + filename)

In [174]:
for index, row in movieInfo[1651:].iterrows():
    movieID = row['itemId']
    movie_title = row['title']
    movie_tagword = row['tagword']
    scrape(movieID, movie_title, movie_tagword)

### 3.3 Working with the collected Tweets.....

In [175]:
# specifying the path to csv files
path = "movie_tweets_collected"
  
# csv files in the path
files = glob.glob(path + "/*.csv")
  
print(len(files))
  
# defining an empty list to store 
# content
data_frame = pd.DataFrame()
content = []
  
# checking all the csv files in the 
# specified path
for filename in files:
    
    # reading content of csv file
#     content.append(filename)
    df = pd.read_csv(filename, index_col=None)
    content.append(df)

    
# converting content to data frame
data_frame = pd.concat(content)

1471


In [176]:
data_frame

,Unnamed: 0,Movie Id,Title,Tagword,description,totaltweets,retweetcount,text,hashtags
0,0,1353,1-900,#1-900,DTI Registered • PH-based kpop online shop • A...,4643,1,#SAPHOngoingGOs\n[PH GO] Agust D - [D-DAY]\n\n...,['SAPHOngoingGOs']
1,1,1353,1-900,#1-900,Spring Day and Ddaeng enthusiast,13589,1,WTS LFB BTS PH\nUntil Today only\n\nJungkook C...,[]
2,2,1353,1-900,#1-900,아포방포 ♡︎ bts fan account .ᐟ,23542,0,🧸 ˗ˏˋ #gelsells_ wts lfb ph only ࿐ྂ\n\n˚ ༘♡ ⋆｡...,['gelsells_']
3,3,1353,1-900,#1-900,We promote lightning safety education and awar...,1948,0,"@P_Ustaz ₦1,900 airtime sent! Thanks for suppo...",[]
4,4,1353,1-900,#1-900,We promote lightning safety education and awar...,1948,0,"@UsmanAina2 ₦1,900 airtime sent! Thanks for su...",[]
...,...,...,...,...,...,...,...,...,...
95,95,232,Young Guns,#youngguns,🇨🇦 Billet dad of champions #GoRaidersGo #TAFC ...,103391,0,"Hair down, bar down! Let’s go #YoungGuns https...",['YoungGuns']
96,96,232,Young Guns,#youngguns,The official Twitter of Oldham Netball Club.,12881,1,The final game of the season sees our amazing ...,"['NWprem', 'youngguns', 'ONCgirls', 'RedAndBlu..."
97,97,232,Young Guns,#youngguns,#MarriedWithChildren fullback who scored 4 tou...,937,10,I love - Young Guns. There is a little bit of ...,[]
98,98,232,Young Guns,#youngguns,🇨🇦 Billet dad of champions #GoRaidersGo #TAFC ...,103391,0,#YoungGuns tie it up early in the 2nd but give...,['YoungGuns']
